In [ ]:
import json
model_type='Flt_ent_role_model'
ver=2

### Wrong prediction Freq and Span analysis Function

In [ ]:

dt_set='tst'
dt_set=dt_set+'_'
project_directory='/content/drive/MyDrive/PHD_assessment_gmu/'
results_filepath=project_directory+'data/'+dt_set+model_type+'_'+str(ver)+'.json'
with open(results_filepath, 'r') as file:
    data = json.load(file)
entity_dict = data['entity']
role_dict=data['role']
method_dict=data['method']
status_dict=data['status']

In [ ]:
# Identifies spans in a document and categorises the spans into wrong and correct spans list
def count_wrong_predictions_in_spans(data):
    '''
    Analyzes labels and identifies a span.
    Records wrong counts of tokens being classified in a span.
    Divides the spans list into all tokens are classified correct and other atleast one wrongly classified.
    Returns all spans present in a doc and correct classified spans and wrong classified spans.
    '''
    all_spans = []
    current_span = []
    wrong_count = 0

    for token, label, prediction in data:
        if label > 0:  # Part of a span
            current_span.append((token, label, prediction))
            if label != prediction:
                wrong_count += 1
        else:  # Span ends
            if current_span:  # If there was a span
                all_spans.append((current_span, wrong_count))
                current_span = []  # Reset for next span
                wrong_count = 0
    # Check for the last span
    if current_span:
      all_spans.append((current_span, wrong_count))
    correct_spans=[]
    wrong_spans=[]
    for span, wrong_cnt in all_spans:
      if wrong_cnt==0:
          correct_spans.append((span,wrong_cnt))
      else:
          wrong_spans.append((span,wrong_cnt))

    return all_spans,correct_spans,wrong_spans

#Calculates wrong prediction frequency at token level
def get_wrong_predictions_freq(entity_dict,id_label_ent):
    """
    Calculates Frequency of each wrong token and also collects the labels of it
    """
    wrong_prediction_freq = {}
    wrong_prediction_lab={}
    for filename,pred_lab in entity_dict.items():
        for token, label, prediction in pred_lab:
            if label != prediction:
                if token not in wrong_prediction_freq:
                    wrong_prediction_freq[token] = 1
                    wrong_prediction_lab[token]=[]
                    wrong_prediction_lab[token].append((id_label_ent[label], id_label_ent[prediction]))
                else:
                    wrong_prediction_freq[token] += 1
                    wrong_prediction_lab[token].append((id_label_ent[label], id_label_ent[prediction]))
    return wrong_prediction_freq,wrong_prediction_lab
def extract_false_positive_spans(data):
    '''
    Analyzes labels and identifies a span.
    Records wrong counts of tokens being classified in a span.
    Divides the spans list into all tokens are classified correct and other atleast one wrongly classified.
    Returns all spans present in a doc and correct classified spans and wrong classified spans.
    '''
    all_spans = []
    current_span = []
    wrong_count = 0

    for token, label, prediction in data:
        if prediction > 0:  # Part of a span
            current_span.append((token, label, prediction))
            if label != prediction:
                wrong_count += 1
        else:  # Span ends
            if current_span:  # If there was a span
                all_spans.append((current_span, wrong_count))
                current_span = []  # Reset for next span
                wrong_count = 0
    # Check for the last span
    if current_span:
      all_spans.append((current_span, wrong_count))
    correct_spans=[]
    wrong_spans=[]
    for span, wrong_cnt in all_spans:
      if wrong_cnt==0:
          correct_spans.append((span,wrong_cnt))
      else:
          wrong_spans.append((span,wrong_cnt))

    return all_spans,correct_spans,wrong_spans

### Entity analysis

In [ ]:
label_id_ent = {'B-Alcohol':1,
 'B-Drug':3,
 'B-Family':5,
 'B-Tobacco':7,
 'I-Alcohol':2,
 'I-Drug':4,
 'I-Family':6,
 'I-Tobacco':8,
 'O':0}
id_label_ent = {v: k for k, v in label_id_ent.items()}

In [ ]:
len(entity_dict)

68

In [ ]:
entity_dict['451_Consult-HistoryandPhy.-RecordsReview-Orthopedic_8']

[['[CLS]', 0, 0],
 ['social', 0, 0],
 ['history', 0, 0],
 [':', 0, 0],
 ['he', 0, 0],
 ['does', 0, 0],
 ['admit', 0, 0],
 ['to', 0, 0],
 ['one', 7, 0],
 ['half', 8, 0],
 ['pack', 8, 0],
 ['of', 8, 0],
 ['cigarette', 8, 0],
 ['consumption', 8, 0],
 ['per', 8, 0],
 ['day', 8, 0],
 ['.', 0, 0],
 ['he', 0, 0],
 ['denies', 0, 0],
 ['any', 0, 0],
 ['alcohol', 1, 1],
 ['consumption', 2, 2],
 ['.', 2, 0],
 ['[SEP]', 0, 0]]

In [ ]:
#Extracting wrong and corrrect spans from test data.
total_dataset_spans=[]
wrong_spans_list=[]
correct_spans_list=[]
for filename,pred_lab in entity_dict.items():
  total_spans,correct_spans,wrong_spans=count_wrong_predictions_in_spans(pred_lab)
  wrong_spans_list.append((filename,wrong_spans))
  correct_spans_list.append((filename,correct_spans))
  total_dataset_spans.append(total_spans)

In [ ]:
len(total_dataset_spans)

68

In [ ]:
#Seperating spans with length one and more
single_span=[]
spans_lst=[]
for ele in wrong_spans_list:
  if ele[1] != []:
    for sp, cnt in ele[1]:
      if len(sp) >1:
        spans_lst.append((ele[0],(sp,cnt)))
      else:
        single_span.append((ele[0],(sp,cnt)))



In [ ]:
{'B-Alcohol':1,
 'B-Drug':3,
 'B-Family':5,
 'B-Tobacco':7,
 'I-Alcohol':2,
 'I-Drug':4,
 'I-Family':6,
 'I-Tobacco':8,
 'O':0}

{'B-Alcohol': 1,
 'B-Drug': 3,
 'B-Family': 5,
 'B-Tobacco': 7,
 'I-Alcohol': 2,
 'I-Drug': 4,
 'I-Family': 6,
 'I-Tobacco': 8,
 'O': 0}

In [ ]:
len(spans_lst)

17

In [ ]:
spans_lst

[('112_Consult-HistoryandPhy.-Consult-RectalBleeding_8',
  ([('quite', 7, 0), ('smoking', 8, 7)], 2)),
 ('259_Consult-HistoryandPhy.-Head&NeckCancerConsult_9',
  ([('does', 1, 0), ('drink', 2, 0), ('alcohol', 2, 1)], 3)),
 ('38_Consult-HistoryandPhy.-BilateralHipPain_8',
  ([('consume', 1, 0), ('##s', 2, 0), ('alcohol', 2, 1)], 3)),
 ('432_Consult-HistoryandPhy.-PsychConsult-Lethargy_11',
  ([('tobacco', 7, 7), (',', 8, 0), ('alcohol', 1, 1)], 1)),
 ('273_Consult-HistoryandPhy.-HypersensitivitytoCoumadin_5',
  ([('one', 5, 0), ('child', 6, 0)], 2)),
 ('184_Consult-HistoryandPhy.-GastricBypassDiscussion-3_7',
  ([('four', 5, 5), ('children', 6, 6), ('.', 6, 0)], 1)),
 ('407_Consult-HistoryandPhy.-PediatricRheumatologyConsult_8',
  ([('sister', 5, 5), (',', 6, 0)], 1)),
 ('457_Consult-HistoryandPhy.-RenalFailureEvaluation_8',
  ([('drug', 3, 3), ('use', 4, 0)], 1)),
 ('326_Consult-HistoryandPhy.-Neck&BackPain_12',
  ([('ill', 3, 0), ('##icit', 4, 0), ('substances', 4, 3)], 3)),
 ('468_Co

In [ ]:
single_span

[('405_Consult-HistoryandPhy.-PainManagementConsult-2_7',
  ([('alcohol', 1, 0)], 1)),
 ('430_Consult-HistoryandPhy.-PsychConsult-Depression-1_9',
  ([('patient', 1, 0)], 1)),
 ('430_Consult-HistoryandPhy.-PsychConsult-Depression-1_9',
  ([('drank', 1, 0)], 1)),
 ("270_Consult-HistoryandPhy.-Huntington'sDisease-Consult_11",
  ([('husband', 5, 0)], 1)),
 ("270_Consult-HistoryandPhy.-Huntington'sDisease-Consult_11",
  ([('drinks', 1, 0)], 1)),
 ('427_Consult-HistoryandPhy.-PsychConsult-AssaultiveBehavior_10',
  ([('wife', 5, 0)], 1)),
 ('427_Consult-HistoryandPhy.-PsychConsult-AssaultiveBehavior_10',
  ([('children', 5, 0)], 1)),
 ('402_Consult-HistoryandPhy.-OtitisMedia-H&P_7', ([('child', 5, 0)], 1)),
 ('321_Consult-HistoryandPhy.-MigrainewithoutAura-Consult_4',
  ([('alcohol', 1, 0)], 1)),
 ('266_Consult-HistoryandPhy.-HipFracture-ERConsult_8', ([('son', 5, 0)], 1)),
 ('325_Consult-HistoryandPhy.-Murmur&Bacteremia._9',
  ([('daughter', 5, 0)], 1))]

In [ ]:
correct_spans_list

[('112_Consult-HistoryandPhy.-Consult-RectalBleeding_8',
  [([('4', 5, 5), ('children', 6, 6)], 0),
   ([('drink', 1, 1), ('alcohol', 2, 2)], 0)]),
 ('116_Consult-HistoryandPhy.-Consult-Sepsis_6',
  [([('smoke', 7, 7), ('##r', 8, 8)], 0),
   ([('cocaine', 3, 3)], 0),
   ([('alcohol', 1, 1)], 0)]),
 ('118_Consult-HistoryandPhy.-Consult-StasisUlcer_7',
  [([('alcohol', 1, 1)], 0), ([('smoke', 7, 7), ('##r', 8, 8)], 0)]),
 ('190_Consult-HistoryandPhy.-GenMedConsult-13_8', [([('smoke', 7, 7)], 0)]),
 ('259_Consult-HistoryandPhy.-Head&NeckCancerConsult_9',
  [([('two', 5, 5), ('adult', 6, 6), ('sons', 6, 6)], 0),
   ([('smoking', 7, 7)], 0),
   ([('drugs', 3, 3)], 0)]),
 ('285_Consult-HistoryandPhy.-IschemicCecum-Consult_8',
  [([('smoking', 7, 7)], 0), ([('alcohol', 1, 1)], 0)]),
 ('303_Consult-HistoryandPhy.-LowerQuadrantPain_10',
  [([('smoke', 7, 7)], 0), ([('drinks', 1, 1)], 0)]),
 ('162_Consult-HistoryandPhy.-ENTConsult_14',
  [([('smoke', 7, 7), ('##r', 8, 8)], 0), ([('alcohol', 1, 1

In [ ]:
for ele in correct_spans_list:
  if ele[0]=='430_Consult-HistoryandPhy.-PsychConsult-Depression-1_9':
    print(ele)

('430_Consult-HistoryandPhy.-PsychConsult-Depression-1_9', [([('drinking', 1, 1)], 0), ([('am', 3, 3), ('##phe', 4, 4), ('##tamine', 4, 4), ('##s', 4, 4)], 0), ([('cocaine', 3, 3)], 0), ([('marijuana', 3, 3)], 0)])


In [ ]:
#Extracting False positives
total_fp_dataset_spans=[]
wrong_fp_spans_list=[]
correct_fp_spans_list=[]
for filename,pred_lab in entity_dict.items():
  total_spans,correct_spans,wrong_spans=extract_false_positive_spans(pred_lab)
  wrong_fp_spans_list.append((filename,wrong_spans))
  correct_fp_spans_list.append((filename,correct_spans))
  total_fp_dataset_spans.append(total_spans)

In [ ]:
single_fp_span=[]
spans_fp_lst=[]
for ele in wrong_fp_spans_list:
  if ele[1] != []:
    for sp, cnt in ele[1]:
      if len(sp) >1:
        spans_fp_lst.append((ele[0],(sp,cnt)))
      else:
        single_fp_span.append((ele[0],(sp,cnt)))


In [ ]:
label_id_ent

{'B-Alcohol': 1,
 'B-Drug': 3,
 'B-Family': 5,
 'B-Tobacco': 7,
 'I-Alcohol': 2,
 'I-Drug': 4,
 'I-Family': 6,
 'I-Tobacco': 8,
 'O': 0}

In [ ]:
spans_fp_lst

[('118_Consult-HistoryandPhy.-Consult-StasisUlcer_7',
  ([('alcohol', 1, 1), ('drinking', 0, 2)], 1)),
 ('406_Consult-HistoryandPhy.-ParoxysmalAtrialFibrillation_10',
  ([('four', 0, 5), ('adult', 5, 6), ('children', 6, 6)], 2)),
 ('185_Consult-HistoryandPhy.-GastrointestinalBleed-ERVisit_10',
  ([('has', 0, 1), ('a', 1, 2), ('drink', 2, 1)], 3))]

In [ ]:
single_fp_span

[('112_Consult-HistoryandPhy.-Consult-RectalBleeding_8',
  ([('smoking', 8, 7)], 1)),
 ('259_Consult-HistoryandPhy.-Head&NeckCancerConsult_9',
  ([('wife', 0, 5)], 1)),
 ('259_Consult-HistoryandPhy.-Head&NeckCancerConsult_9',
  ([('alcohol', 2, 1)], 1)),
 ('324_Consult-HistoryandPhy.-MultipleNeurologicalSymptoms_8',
  ([('wife', 0, 5)], 1)),
 ('38_Consult-HistoryandPhy.-BilateralHipPain_8', ([('alcohol', 2, 1)], 1)),
 ('405_Consult-HistoryandPhy.-PainManagementConsult-2_7',
  ([('drinks', 0, 1)], 1)),
 ('419_Consult-HistoryandPhy.-PreopCardiacConsult_9', ([('smoke', 0, 7)], 1)),
 ('419_Consult-HistoryandPhy.-PreopCardiacConsult_9',
  ([('daughter', 0, 5)], 1)),
 ('419_Consult-HistoryandPhy.-PreopCardiacConsult_9',
  ([('grandson', 0, 5)], 1)),
 ('28_Consult-HistoryandPhy.-AttemptedSuicide-Consult_10',
  ([('drug', 0, 3)], 1)),
 ('124_Consult-HistoryandPhy.-Consult_ERReport-OB_GYN_8',
  ([('partner', 0, 5)], 1)),
 ('124_Consult-HistoryandPhy.-Consult_ERReport-OB_GYN_8',
  ([('father', 0

In [ ]:
#Extracting the wrong tokens frequency
ent_wrng_pred_frq,ent_wrng_pred_lab=get_wrong_predictions_freq(entity_dict,id_label_ent)

In [ ]:
sorted(ent_wrng_pred_frq)

['##icit',
 '##s',
 ',',
 '.',
 '21',
 '23',
 '24',
 'a',
 'adult',
 'alcohol',
 'beers',
 'both',
 'boyfriend',
 'child',
 'children',
 'cigarette',
 'consume',
 'consumption',
 'daughter',
 'day',
 'does',
 'drank',
 'drink',
 'drinking',
 'drinks',
 'drug',
 'father',
 'four',
 'grandson',
 'half',
 'has',
 'husband',
 'ill',
 'of',
 'one',
 'pack',
 'parents',
 'partner',
 'patient',
 'per',
 'quite',
 'siblings',
 'smoke',
 'smoking',
 'son',
 'substances',
 'use',
 'wife']

## Role level Analysis

In [ ]:
id_label_role={0:'O',1:'B-Type',2:'I-Type',3:'B-Amount',4:'I-Amount',5:'B-Temporal',6:'I-Temporal',7:'B-Frequency',8:'I-Frequency',9:'B-QuitHistory',10:'I-QuitHistory',11:'B-ExposureHistory',12:'I-ExposureHistory',13:'B-Location',14:'I-Location'}


In [ ]:
#Extracting wrong and corrrect spans from test data.
total_role_dataset_spans=[]
wrong_role_spans_list=[]
correct_role_spans_list=[]
for filename,pred_lab in role_dict.items():
  total_spans,correct_spans,wrong_spans=count_wrong_predictions_in_spans(pred_lab)
  wrong_role_spans_list.append((filename,wrong_spans))
  correct_role_spans_list.append((filename,correct_spans))
  total_role_dataset_spans.append(total_spans)

In [ ]:
#Seperating spans with length one and more

single_role_span=[]
spans_role_lst=[]
for ele in wrong_role_spans_list:
  if ele[1] != []:
    for sp, cnt in ele[1]:
      if len(sp) >1:
        spans_role_lst.append((ele[0],(sp,cnt)))
      else:
        single_role_span.append((ele[0],(sp,cnt)))



In [ ]:
id_label_role={0:'O',1:'B-Type',2:'I-Type',3:'B-Amount',4:'I-Amount',5:'B-Temporal',6:'I-Temporal',7:'B-Frequency',8:'I-Frequency',9:'B-QuitHistory',10:'I-QuitHistory',11:'B-ExposureHistory',12:'I-ExposureHistory',13:'B-Location',14:'I-Location'}


In [ ]:
spans_role_lst

[('112_Consult-HistoryandPhy.-Consult-RectalBleeding_8',
  ([('4', 3, 3), ('children', 2, 1)], 1)),
 ('112_Consult-HistoryandPhy.-Consult-RectalBleeding_8',
  ([('35', 11, 12),
    ('-', 12, 12),
    ('year', 12, 12),
    ('history', 12, 0),
    ('of', 12, 0),
    ('smoking', 12, 0)],
   4)),
 ('118_Consult-HistoryandPhy.-Consult-StasisUlcer_7',
  ([('several', 9, 10), ('years', 10, 10), ('ago', 10, 10)], 1)),
 ('118_Consult-HistoryandPhy.-Consult-StasisUlcer_7',
  ([('several', 9, 10), ('years', 10, 10), ('ago', 10, 10)], 1)),
 ('190_Consult-HistoryandPhy.-GenMedConsult-13_8',
  ([('in', 9, 0), ('1985', 10, 10)], 1)),
 ('259_Consult-HistoryandPhy.-Head&NeckCancerConsult_9',
  ([('has', 1, 0),
    ('been', 2, 0),
    ('separated', 2, 0),
    ('from', 2, 0),
    ('his', 2, 0),
    ('wife', 2, 1),
    ('for', 5, 6),
    ('many', 6, 6),
    ('years', 6, 6)],
   7)),
 ('259_Consult-HistoryandPhy.-Head&NeckCancerConsult_9',
  ([('two', 3, 3), ('adult', 1, 1), ('sons', 2, 1)], 1)),
 ('259_Co

In [ ]:
role_wrng_pred_frq,role_wrng_pred_lab=get_wrong_predictions_freq(role_dict,id_label_role)

In [ ]:

dict(sorted(role_wrng_pred_frq.items(), key=lambda item: item[1], reverse=True))


{'.': 9,
 'for': 8,
 'of': 5,
 'a': 5,
 'children': 4,
 'home': 4,
 'to': 4,
 'married': 3,
 'smoking': 3,
 'in': 3,
 'and': 3,
 'years': 3,
 'history': 2,
 'several': 2,
 'separated': 2,
 'wife': 2,
 'ma': 2,
 'j': 2,
 'work': 2,
 'daily': 2,
 'at': 2,
 'the': 2,
 'her': 2,
 'daughter': 2,
 'five': 2,
 'ago': 2,
 'on': 2,
 '-': 2,
 'abuse': 2,
 ',': 2,
 'domestic': 2,
 'b': 2,
 '##al': 2,
 '##ti': 2,
 '##more': 2,
 '##maker': 2,
 'four': 2,
 'week': 2,
 '3': 2,
 'locally': 2,
 '##ado': 2,
 '35': 1,
 'has': 1,
 'been': 1,
 'from': 1,
 'his': 1,
 'sons': 1,
 'air': 1,
 'force': 1,
 'lock': 1,
 '##hee': 1,
 '##d': 1,
 '##rt': 1,
 '##in': 1,
 '##ul': 1,
 '##y': 1,
 '27': 1,
 'long': 1,
 'beer': 1,
 '6': 1,
 'pipeline': 1,
 'about': 1,
 '20': 1,
 'monthly': 1,
 'low': 1,
 'play': 1,
 'softball': 1,
 'quit': 1,
 '5': 1,
 'time': 1,
 'bypass': 1,
 'surgery': 1,
 'grandson': 1,
 'running': 1,
 'restaurant': 1,
 'husband': 1,
 'se': 1,
 '##da': 1,
 '##tive': 1,
 'rarely': 1,
 'off': 1,
 'basis

## Status level Analysis

In [ ]:
id_label_status={0:'O',1:'B-Status',2:'I-Status'}

In [ ]:
#Extracting wrong and corrrect spans from test data.
total_status_dataset_spans=[]
wrong_status_spans_list=[]
correct_status_spans_list=[]
for filename,pred_lab in status_dict.items():
  total_spans,correct_spans,wrong_spans=count_wrong_predictions_in_spans(pred_lab)
  wrong_status_spans_list.append((filename,wrong_spans))
  correct_status_spans_list.append((filename,correct_spans))
  total_status_dataset_spans.append(total_spans)

In [ ]:
#Seperating spans with length one and more

single_status_span=[]
spans_status_lst=[]
for ele in wrong_status_spans_list:
  if ele[1] != []:
    for sp, cnt in ele[1]:
      if len(sp) >1:
        spans_status_lst.append((ele[0],(sp,cnt)))
      else:
        single_status_span.append((ele[0],(sp,cnt)))



In [ ]:
whole_span_wrong=[]
partially_correct=[]
for ele in spans_status_lst:
  len_spans=len(ele[1][0])
  if len_spans == ele[1][1]:
    whole_span_wrong.append(ele)
  elif len_spans > ele[1][1]:
    partially_correct.append(ele)

In [ ]:
whole_span_wrong

[('116_Consult-HistoryandPhy.-Consult-Sepsis_6',
  ([('in', 1, 0), ('the', 2, 0), ('past', 2, 0)], 3)),
 ('285_Consult-HistoryandPhy.-IschemicCecum-Consult_8',
  ([('have', 1, 0), ('chronic', 2, 0)], 2)),
 ('162_Consult-HistoryandPhy.-ENTConsult_14',
  ([('at', 1, 0), ('this', 2, 0), ('time', 2, 0)], 3)),
 ('162_Consult-HistoryandPhy.-ENTConsult_14',
  ([('is', 1, 0), ('a', 2, 0)], 2)),
 ('38_Consult-HistoryandPhy.-BilateralHipPain_8',
  ([('uses', 1, 0), ('no', 2, 1)], 2)),
 ('38_Consult-HistoryandPhy.-BilateralHipPain_8',
  ([('likes', 1, 0), ('to', 2, 0), ('play', 2, 0)], 3)),
 ('28_Consult-HistoryandPhy.-AttemptedSuicide-Consult_10',
  ([('actively', 1, 0),
    ('to', 2, 0),
    ('##x', 2, 0),
    ('screen', 2, 0),
    ('positive', 2, 0)],
   5)),
 ('273_Consult-HistoryandPhy.-HypersensitivitytoCoumadin_5',
  ([('lived', 1, 0), ('in', 2, 0)], 2)),
 ('273_Consult-HistoryandPhy.-HypersensitivitytoCoumadin_5',
  ([('has', 1, 0), ('been', 2, 0), ('predominantly', 2, 0)], 3)),
 ('273_Co

In [ ]:
partially_correct

[('112_Consult-HistoryandPhy.-Consult-RectalBleeding_8',
  ([('is', 1, 1), ('married', 2, 0)], 1)),
 ('190_Consult-HistoryandPhy.-GenMedConsult-13_8',
  ([('is', 1, 1), ('married', 2, 0)], 1)),
 ('190_Consult-HistoryandPhy.-GenMedConsult-13_8',
  ([('is', 1, 1), ('retired', 2, 0)], 1)),
 ('259_Consult-HistoryandPhy.-Head&NeckCancerConsult_9',
  ([('reports', 1, 0),
    ('quit', 2, 1),
    ('##ting', 2, 2),
    ('on', 2, 0),
    ('j', 2, 0),
    ('##ul', 2, 0),
    ('##y', 2, 0),
    ('27', 2, 0)],
   7)),
 ('319_Consult-HistoryandPhy.-MetastaticOvarianCancer-Consult_10',
  ([('quit', 1, 1), ('smoking', 2, 0)], 1)),
 ('319_Consult-HistoryandPhy.-MetastaticOvarianCancer-Consult_10',
  ([('is', 1, 1), ('married', 2, 0)], 1)),
 ('324_Consult-HistoryandPhy.-MultipleNeurologicalSymptoms_8',
  ([('previously', 1, 1), ('worked', 2, 0)], 1)),
 ('38_Consult-HistoryandPhy.-BilateralHipPain_8',
  ([('is', 1, 1), ('married', 2, 0)], 1)),
 ('419_Consult-HistoryandPhy.-PreopCardiacConsult_9',
  ([('i

In [ ]:
spans_status_lst

[('112_Consult-HistoryandPhy.-Consult-RectalBleeding_8',
  ([('is', 1, 1), ('married', 2, 0)], 1)),
 ('116_Consult-HistoryandPhy.-Consult-Sepsis_6',
  ([('in', 1, 0), ('the', 2, 0), ('past', 2, 0)], 3)),
 ('190_Consult-HistoryandPhy.-GenMedConsult-13_8',
  ([('is', 1, 1), ('married', 2, 0)], 1)),
 ('190_Consult-HistoryandPhy.-GenMedConsult-13_8',
  ([('is', 1, 1), ('retired', 2, 0)], 1)),
 ('259_Consult-HistoryandPhy.-Head&NeckCancerConsult_9',
  ([('reports', 1, 0),
    ('quit', 2, 1),
    ('##ting', 2, 2),
    ('on', 2, 0),
    ('j', 2, 0),
    ('##ul', 2, 0),
    ('##y', 2, 0),
    ('27', 2, 0)],
   7)),
 ('285_Consult-HistoryandPhy.-IschemicCecum-Consult_8',
  ([('have', 1, 0), ('chronic', 2, 0)], 2)),
 ('162_Consult-HistoryandPhy.-ENTConsult_14',
  ([('at', 1, 0), ('this', 2, 0), ('time', 2, 0)], 3)),
 ('162_Consult-HistoryandPhy.-ENTConsult_14',
  ([('is', 1, 0), ('a', 2, 0)], 2)),
 ('319_Consult-HistoryandPhy.-MetastaticOvarianCancer-Consult_10',
  ([('quit', 1, 1), ('smoking', 

In [ ]:
single_status_span

[('112_Consult-HistoryandPhy.-Consult-RectalBleeding_8',
  ([('retired', 1, 0)], 1)),
 ('112_Consult-HistoryandPhy.-Consult-RectalBleeding_8',
  ([('quite', 1, 0)], 1)),
 ('190_Consult-HistoryandPhy.-GenMedConsult-13_8', ([('quit', 1, 0)], 1)),
 ('259_Consult-HistoryandPhy.-Head&NeckCancerConsult_9',
  ([('retired', 1, 0)], 1)),
 ('285_Consult-HistoryandPhy.-IschemicCecum-Consult_8', ([('had', 1, 0)], 1)),
 ('319_Consult-HistoryandPhy.-MetastaticOvarianCancer-Consult_10',
  ([('occasionally', 1, 0)], 1)),
 ('419_Consult-HistoryandPhy.-PreopCardiacConsult_9', ([('quit', 1, 0)], 1)),
 ('419_Consult-HistoryandPhy.-PreopCardiacConsult_9', ([('is', 1, 0)], 1)),
 ('430_Consult-HistoryandPhy.-PsychConsult-Depression-1_9',
  ([('history', 1, 0)], 1)),
 ('236_Consult-HistoryandPhy.-GenMedConsult-6_7', ([('widowed', 1, 0)], 1)),
 ('236_Consult-HistoryandPhy.-GenMedConsult-6_7', ([('home', 1, 0)], 1)),
 ('236_Consult-HistoryandPhy.-GenMedConsult-6_7', ([('retired', 1, 0)], 1)),
 ("270_Consult-His

In [ ]:
status_wrng_pred_frq,status_wrng_pred_lab=get_wrong_predictions_freq(status_dict,id_label_status)

In [ ]:

dict(sorted(status_wrng_pred_frq.items(), key=lambda item: item[1], reverse=True))


{'is': 15,
 'married': 7,
 'to': 6,
 'has': 6,
 'retired': 5,
 'history': 5,
 'a': 4,
 'not': 4,
 'in': 3,
 'quit': 3,
 'was': 3,
 'of': 3,
 'have': 3,
 'occasionally': 3,
 'been': 3,
 'drinks': 3,
 'use': 3,
 'smoke': 3,
 'the': 2,
 'past': 2,
 'used': 2,
 'smoking': 2,
 'uses': 2,
 'no': 2,
 'widowed': 2,
 'denies': 2,
 'lives': 2,
 'any': 2,
 'quite': 1,
 'reports': 1,
 'on': 1,
 'j': 1,
 '##ul': 1,
 '##y': 1,
 '27': 1,
 'had': 1,
 'chronic': 1,
 'at': 1,
 'this': 1,
 'time': 1,
 'work': 1,
 'worked': 1,
 'likes': 1,
 'play': 1,
 'smoked': 1,
 'sober': 1,
 'home': 1,
 'be': 1,
 'actively': 1,
 '##x': 1,
 'screen': 1,
 'positive': 1,
 'employed': 1,
 'currently': 1,
 'originally': 1,
 'lived': 1,
 'predominantly': 1,
 'done': 1,
 'some': 1,
 'served': 1,
 'he': 1,
 'only': 1,
 'live': 1,
 'single': 1,
 'born': 1,
 'alcohol': 1,
 'does': 1,
 'ever': 1,
 'admits': 1,
 'using': 1,
 '##rs': 1,
 'drink': 1,
 '##er': 1,
 'none': 1}

## Method level Analysis

In [ ]:
id_label_method={0:'O',1:'B-Method',2:'I-Method'}

In [ ]:
#Extracting wrong and corrrect spans from test data.
total_method_dataset_spans=[]
wrong_method_spans_list=[]
correct_method_spans_list=[]
for filename,pred_lab in method_dict.items():
  total_spans,correct_spans,wrong_spans=count_wrong_predictions_in_spans(pred_lab)
  wrong_method_spans_list.append((filename,wrong_spans))
  correct_method_spans_list.append((filename,correct_spans))
  total_method_dataset_spans.append(total_spans)

In [ ]:
#Seperating spans with length one and more

single_method_span=[]
spans_method_lst=[]
for ele in wrong_method_spans_list:
  if ele[1] != []:
    for sp, cnt in ele[1]:
      if len(sp) >1:
        spans_method_lst.append((ele[0],(sp,cnt)))
      else:
        single_method_span.append((ele[0],(sp,cnt)))



In [ ]:
spans_method_lst

[('259_Consult-HistoryandPhy.-Head&NeckCancerConsult_9',
  ([('born', 1, 0), ('and', 2, 0), ('raised', 2, 1)], 3)),
 ('324_Consult-HistoryandPhy.-MultipleNeurologicalSymptoms_8',
  ([('on', 1, 0), ('disability', 2, 2)], 1)),
 ('432_Consult-HistoryandPhy.-PsychConsult-Lethargy_11',
  ([('lives', 1, 1),
    ('with', 2, 2),
    ('husband', 2, 2),
    ('of', 2, 0),
    ('40', 2, 0),
    ('years', 2, 0)],
   3)),
 ('427_Consult-HistoryandPhy.-PsychConsult-AssaultiveBehavior_10',
  ([('currently', 1, 0), ('on', 2, 0), ('medical', 2, 2), ('leave', 2, 2)],
   2)),
 ("514_Consult-HistoryandPhy.-Worker'sCompensationInjury_9",
  ([('##smo', 1, 0), ('##ker', 2, 0)], 2)),
 ('468_Consult-HistoryandPhy.-SickSinusSyndrome_6',
  ([('full', 1, 2), ('time', 2, 2)], 1)),
 ('304_Consult-HistoryandPhy.-LumbarRadiculopathy-Consult_4',
  ([('does', 1, 0),
    ('not', 2, 0),
    ('have', 2, 0),
    ('support', 2, 0),
    ('at', 2, 0),
    ('home', 2, 0)],
   6)),
 ('510_Consult-HistoryandPhy.-Well-ChildCheck-5

In [ ]:
single_method_span

[]

In [ ]:
method_wrng_pred_frq,method_wrng_pred_lab=get_wrong_predictions_freq(method_dict,id_label_method)

In [ ]:

dict(sorted(method_wrng_pred_frq.items(), key=lambda item: item[1], reverse=True))


{'and': 5,
 '.': 5,
 'born': 4,
 '-': 4,
 'raised': 3,
 'on': 3,
 'at': 3,
 'separated': 2,
 'from': 2,
 'his': 2,
 'wife': 2,
 'home': 2,
 ',': 2,
 'support': 2,
 'retired': 1,
 'close': 1,
 'off': 1,
 'of': 1,
 '40': 1,
 'years': 1,
 'currently': 1,
 'her': 1,
 'partner': 1,
 'father': 1,
 '21': 1,
 'month': 1,
 'old': 1,
 'daughter': 1,
 'est': 1,
 '##ranged': 1,
 'boyfriend': 1,
 '##smo': 1,
 '##ker': 1,
 'accompanied': 1,
 'with': 1,
 'full': 1,
 'does': 1,
 'not': 1,
 'have': 1,
 'lives': 1,
 'locally': 1,
 'who': 1,
 'is': 1,
 'a': 1,
 'supervisor': 1,
 'ex': 1,
 '##cel': 1}